# Incubyte

I own a Multi-Specialty Hospital chain with locations all across the world. My hospital is famous for Vaccination. Patients who come to my hospital (across the globe) will be given a User Card with which they can access any of my hospital in the world. Current Status: We maintain all customers in one database. There are heaps of customers with user cards to my hospital. So, I decided to split up the customers based on the country and load them into corresponding country tables. To pull the customers as per Country, my developers should know what are all the places the Customer Data is available. So, the data extracting will be done by our Source System. They will pull the all the relevant customer data and will give us a Data file.

Below code perform basic ETL operation and we can improve this code to take care of Edge cases as well

## My approach

1. Here I am using CSV File as source system and SQL database as Destination system.
2. Requirements: -Jupyter note book to write python script and ssms for sql storage.
3. Python script includes Object Oriented concept
4. I have created class hospital which includes few methods to perform ETL work
5. Init function will help us to connect to the database and i am passing Servername and database name as parameters.
6. The create_table method table will help us to create new intermediate table/staging table where we load all the data from Source system.
7. The bulk_insert  method/function will ingest the data to the created staging table.
8. The Countrytable will perform few operations.
   1. Fetch distinct country names from staging table
   2. Loading country names to the list
   3. Creates the separate table for each country in the list and will insert data of specific country data to the table

In [19]:
import pyodbc                              

In [17]:
class Hospital:
    def __init__(self,Server, Database):
        conn = pyodbc.connect('Driver={SQL Server};'+
                      'Server={0};'.format(Server)+
                      'Database={0};'.format(Database)+
                      'Trusted_Connection=yes;')
        conn.autocommit=True
        self.cursor = conn.cursor()
    
    def create_table(self):
        self.cursor.execute('DROP TABLE IF EXISTS stagingtable'.format())
        self.cursor.execute(""" Create table stagingtable(Customerid varchar(MAX), Customername varchar(MAX), CustomerOpendate datetime, LastConsulteddate datetime, VaccinationType CHAR(100), Doctoreconsulted CHAR(100),State CHAR(100),Country CHAR(100),Postal_code INT,DOB datetime,Activecustomer CHAR(100))""".format())
    
   
    def bulk_insert(self,file_path):
            self.cursor.execute("""BULK INSERT stagingtable FROM "{0}" WITH (FIRSTROW = 2,FIELDTERMINATOR='|',ROWTERMINATOR='\\n');""".format(file_path))
    
    
    
    def Countrytable(self):
        countrylist=[]
        self.cursor.execute('select distinct Country from dbo.stagingtable'.format())
        result = self.cursor.fetchall()
        for row in result:
            countrylist.append(row[0])
        for i in range(len(countrylist)):
            self.cursor.execute('DROP TABLE IF EXISTS [{0}{1}]'.format("Country_",countrylist[i]))
            self.cursor.execute("SELECT Customerid,Customername, FORMAT([CustomerOpendate],'yyyyMMdd') as 'CustomerOpendate',FORMAT([LastConsulteddate],'yyyyMMdd') as 'LastConsulteddate',VaccinationType,Doctoreconsulted,State,Country,Postal_code, FORMAT([DOB],'yyyyMMdd') as 'DOB', Activecustomer INTO  [{0}{1}] FROM stagingtable WHERE Country='{1}'".format("Country_",countrylist[i]))
             

In [18]:
hopitaldatabase = Hospital("Servername", "Hospital") #reply Server name with Actual Server
hopitaldatabase.create_table()
file_path="C:\\Users\\chand\\Desktop\\Assignment\\Hospital_database\\mydata.csv" # replace filepath
hopitaldatabase.bulk_insert(file_path)
hopitaldatabase.Countrytable()

# END